## NoiseData training
- 데이터가 너무 커서 8개로 split   
> 결론적으로는 실패 (추후에 하게 된다면 데이터 생성부터 다시)
- 노이즈가 있는 폰트 이미지 추가  

### 기본 옵션

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import os, glob, sys
import numpy as np
import pandas as pd
from PIL import Image
import pickle
import cv2
import matplotlib.pyplot as plt

import tensorflow
import keras

from keras.models import load_model
from tensorflow.compat.v2.keras.models import model_from_json

Using TensorFlow backend.


### model, weight, data load

In [ ]:
# pickle파일 read
with open("/content/drive/My Drive/KPMG/classifier/pickle/batang_noise.pkl","rb") as pk:
  batang_pk = pickle.load(pk)
with open("/content/drive/My Drive/KPMG/classifier/pickle/dotum_noise.pkl","rb") as pk:
  dotum_pk = pickle.load(pk)
with open("/content/drive/My Drive/KPMG/classifier/pickle/gothic_noise.pkl","rb") as pk:
  gothic_pk = pickle.load(pk)
with open("/content/drive/My Drive/KPMG/classifier/pickle/myeongjo_noise.pkl","rb") as pk:
  myeongjo_pk = pickle.load(pk)

# 전체 이미지 array 저장해놓은 것
# image_all = batang_pk+dotum_pk+gothic_pk+myeongjo_pk

batang_pk=np.array(batang_pk)
dotum_pk=np.array(dotum_pk)
gothic_pk=np.array(gothic_pk)
myeongjo_pk=np.array(myeongjo_pk)

In [ ]:
# 모델, weight 로드
json_file = open("/content/drive/My Drive/KPMG/classifier/model/fc_model_8.json", "r")
loaded_model_json = json_file.read()
json_file.close()

# json파일로부터 model load
loaded_model = model_from_json(loaded_model_json)

# model에 weight 로드하기
weight = '/content/drive/My Drive/KPMG/classifier/model/fc_weight_8.h5'
loaded_model.load_weights(weight)

### data split

In [ ]:
# seed설정
np.random.seed(0)
split_cnt=4

# 인덱스 만들어 주고 순서를 랜덤으로 바꾸어 인덱스를 출력해주는 함수
def make_index_permutation(font_pk):
  result=np.random.permutation(np.arange(len(font_pk)))
  return result

batang_index=make_index_permutation(batang_pk)
dotum_index=make_index_permutation(dotum_pk)
gothic_index=make_index_permutation(gothic_pk)
myeongjo_index=make_index_permutation(myeongjo_pk)

In [ ]:
import math

## font_index를 slicing으로 split 해주고 리스트에 추가해주는 함수

def data_split_merge(split_cnt,font_index):
  result=[]
  for i in range(0,split_cnt):
    result.append(font_index[round((i*(len(font_index))/split_cnt)):round((i+1)*(len(font_index))/split_cnt)])
  return result

batang_index_list=data_split_merge(split_cnt,batang_index)
dotum_index_list=data_split_merge(split_cnt,dotum_index)
gothic_index_list=data_split_merge(split_cnt,gothic_index)
myeongjo_index_list=data_split_merge(split_cnt,myeongjo_index)


In [ ]:
# 원하는 갯수로 train data로 나눠주는 함수


def make_train_data(split_cnt,font_pk,font_index_list):
  result=[]
  for i in range(split_cnt):
    result.append(font_pk[font_index_list[i]])
  return result


In [ ]:
batang_train=make_train_data(split_cnt,batang_pk,batang_index_list)
dotum_train=make_train_data(split_cnt,dotum_pk,dotum_index_list)
gothic_train=make_train_data(split_cnt,gothic_pk,gothic_index_list)
myeongjo_train=make_train_data(split_cnt,myeongjo_pk,myeongjo_index_list)

In [ ]:
# 여기 연산 들어가면서 터짐
# jupyter notebook에서도 안돌아감
# 데이터셋을 2~3번 분할해서 생성 후 돌리기

total_train=[]
for i in range(split_cnt):
  total_train.append(batang_train[i].tolist()+dotum_train[i].tolist()+gothic_train[i].tolist()+myeongjo_train[i].tolist())

In [ ]:
total_df_list=[]

for i in range(split_cnt):
  batang_df= pd.DataFrame({'image':batang_train[i], 'category':0})
  dotum_df = pd.DataFrame({'image':dotum_train[i], 'category':1})
  gothic_df = pd.DataFrame({'image':gothic_train[i], 'category':2})
  myeongjo_df = pd.DataFrame({'image':myeongjo_train[i], 'category':3})
  total_df=pd.concat([batang_df,dotum_df,gothic_df,myeongjo_df])
  total_df.reset_index(drop=True, inplace=True)
  total_df_list.append(total_df)



In [ ]:
X_list=[]

def make_X_list(total_train_data):
  img=[]
  for i in range(len(total_train_data)):
    img.append(cv2.resize(total_train_data[i],(64,64)))
  return img

for i in range(split_cnt):
  X_list.append(make_X_list(total_train[i]))

### 노이즈있는 데이터 추가 학습

##### 1번째 학습

In [ ]:
def make_train_XY(X,total_df_list,idx):
  return X_list[idx],np.array(total_df_list[idx]['category'])

#### 아래 train_XY안에 있는 idx를 0~split_cnt-1까지 바꿔주면서 넣어주기만 하면 됨

X,Y=make_train_XY(X_list,total_df_list,0)
X=np.array(X)

# Train, valid set split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, 
                                                    stratify=Y ,random_state=1234)

In [ ]:
loaded_model.compile(loss="sparse_categorical_crossentropy", optimizer="rmsprop", metrics=["accuracy"])

def Time(model, train_X,train_Y, batch_size, n_epoch):

  training_start_time = time()
  trained_model = model.fit(train_X, train_Y,
                            batch_size=batch_size, epochs=n_epoch,
                            validation_split = 0.2)
  training_end_time = time()
  print("Training time finished.\n%d epochs in %12.2f" % (n_epoch,
                                                          training_end_time - training_start_time))
  return trained_model

fitted_model=Time(loaded_model,X_train,y_train,batch_size=500,n_epoch=20)